# **Lab 15: AI Ethics**
---

### **Description**:
In this lab, we will analyze UCI's Adult Data set and explore ethical issues with the dataset and models created from it, as well as attempt to mitigate any bias we find. Then, we will explore Amazon's Clarify.

<br>

### **Lab Structure**
**Part 1**: [Using Visualizations to Identify Bias](#p1)

**Part 2**: [Using AWS Clarify to Identify Bias](#p2)



</br>


### **Goals:**
By the end of this lab, you will be able to:
* Explain why imbalanced data can lead to bias in models.
* You will be able to identify bias through different means like data exploration and using AWS Clarify.
* Mitigate bias in data once you have found it using several different approaches.


<br>

### **Cheat Sheets**
[AWS Clarify](https://docs.google.com/document/d/1eGmQBEzCt4YBgPDxWB7j6UPb1HPucIISz3k1FLGunr4/edit?usp=sharing)

<br>

**Before starting, run the code below to import all necessary functions and libraries.**


In [ ]:
#!pip install scikit-learn
!pip install --quiet smclarify

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from smclarify.bias.report import *

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

<a name="p1"></a>

---
## **Part 1: Using Visualizations to Identify Bias**
---

We will be exploring UCI's [Adult Data Set](https://archive-beta.ics.uci.edu/ml/datasets/adult) and the bias inherent within it. This data was intended to inspect incomes over $50K based on census data in 1994, but as we will discover, there are some pitfalls with this data set. The following table will provide a description of what each column in the data set represents.

<br>

**NOTE:** This data set was derived from 'Census Data'. For this project, we are assuming this refers to the US Census Data for the year 1994. Our comparisons for visualizations reflect this assumption.

<br>

**Run the cell below to load the data.**

In [ ]:
url = "https://raw.githubusercontent.com/batuhansahincanel/UCIsAdultDataset/main/adult.data" #"https://raw.githubusercontent.com/batuhansahincanel/UCIsAdultDataset/main/adult.data"
names=["Age", "Workclass", "Final-Weight", "Education", "Education-number-of-years", "Marital-status",
        "Occupation", "Relationship", "Race", "Sex", "Capital-gain", "Capital-loss",
        "Hours-per-week", "Native-country", "Target"]

adult = pd.read_csv(url, names = names)
adult = adult.dropna()
adult.head()

### **Problem #1.1: Sex Distribution**

Complete the code below to create a bar graph for the sex distribution in this dataset. What do you notice in the visualization?

In [ ]:
sex_labels = adult["Sex"].# COMPLETE THIS LINE
sex_counts = adult["Sex"].# COMPLETE THIS LINE

plt.# COMPLETE THIS LINE

plt.title('Sex Distribution in the Data', fontweight = 'bold')
plt.xlabel('Sex', fontweight = 'bold')
plt.ylabel('Number of Adults', fontweight = 'bold')

plt.show()

#### **Follow Up**

You may have noticed that there is approximately double the amount of males than females in our data. This is a big red flag. If the data were truly representative of the US population in 1994, there would actually be 50.9% females compared to the 49.1% of males.

### **Problem #1.2: Race Distribution**

Complete the code below to create a bar graph for the race distribution in this dataset. What do you notice in the visualization?

<br>

Consider using the following when creating your graph to make the x-ticks more readable: `plt.xticks(rotation = 45)`.

In [ ]:
race_labels = # COMPLETE THIS LINE
race_counts = # COMPLETE THIS LINE

plt.# COMPLETE THIS LINE

plt.title(# COMPLETE THIS LINE
plt.xlabel(# COMPLETE THIS LINE
plt.ylabel(# COMPLETE THIS LINE
plt.xticks(# COMPLETE THIS LINE

plt.show()

### **Problem #1.3: Education Distribution**

Create a bar graph for the education distribution in this dataset. What do you notice in the visualization?

In [ ]:
# COMPLETE THIS CODE

#### **NOTE**

The number of highschool graduates is more than 33% compared to college graduates and almost 50% more compared to individuals with bachelor's degrees.

### **Problem #1.4: Education Distribution Revised**

Run the code cell below to modify your solution from above so that the counts for:
* `1st-4th` and `5th-6th` are grouped under a new label: `Elementary`.
* `9th`, `10th`, `11th`, `12th` are grouped under a new label: `High School`.

In [ ]:
education_dict = adult["Education"].value_counts().to_dict()

# Group all elementary
education_dict["Elementary"] = 0
for value in [' 1st-4th', ' 5th-6th']:
    education_dict["Elementary"] += education_dict.pop(value)

# Group all High School
education_dict["High School"] = 0
for value in [' 9th', ' 10th', ' 11th', ' 12th']:
    education_dict["High School"] += education_dict.pop(value)

education_labels = education_dict.keys()
education_counts = education_dict.values()


plt.bar(education_labels, education_counts, color = 'lightblue')

plt.title('Education Levels in the Data', fontweight = 'bold')
plt.xlabel('Education Level', fontweight = 'bold')
plt.ylabel('Number of Adults', fontweight = 'bold')
plt.xticks(rotation = 45)

plt.show()

### **Problem #1.5: Income Distribution**


Create a bar graph for the target (income above or below 50K). What do you notice?

In [ ]:
target_labels = adult["Target"].unique()
target_counts = adult["Target"].value_counts()

# COMPLETE THIS CODE

#### **NOTE**

While we do not have as much information about the exact values of earnings for the year 1994, we must remember that this is our target data. It would be ideal to have equal amounts of both categories for our model.

### **Problem #1.6: Target by Sex**

Create a new feature, `Target by Sex`, that designates any person with:
* `Target` of `' <=50K'` and `Sex` of `' Male'` as `'Male <=50K'`.
* `Target` of `' <=50K'` and `Sex` of `' Female'` as `'Female <=50K'`.
* `Target` of `' >50K'` and `Sex` of `' Male'` as `'Male >50K'`.
* `Target` of `' >50K'` and `Sex` of `' Female'` as `'Female >50K'`.


Then create a bar plot of this feature.

In [ ]:
adult.loc[(adult['Target'] == ' <=50K') & (adult['Sex'] == ' Male'), 'Target by Sex'] = 'Male <=50K'
adult.loc[(adult['Target'] == ' <=50K') & (adult['Sex'] == ' Female'), 'Target by Sex'] = 'Female <=50K'
adult.loc[(adult['Target'] == ' >50K') & (adult['Sex'] == ' Male'), 'Target by Sex'] = 'Male >50K'
adult.loc[(adult['Target'] == ' >50K') & (adult['Sex'] == ' Female'), 'Target by Sex'] = 'Female >50K'

target_by_sex_labels = adult["Target by Sex"].unique()
target_by_sex_counts = adult["Target by Sex"].value_counts()

# COMPLETE THIS CODE

#### **NOTE**

This visual is extremely important. If we were to use sex in our model, we could easily be overfitting to the female population since our dataset for that section is much smaller than the rest of the data. Furthermore, there is not even 2,000 data points for females earning over 50K (less than 6% of our data).

<a name="p2"></a>

---
## **Part 2: Using AWS Clarify to Identify Bias**
---

In this section, we are going to use Clarify to identify bias. This method is *much faster* than creating bar graphs for every column.

Here is an AWS Clarify [cheatsheet](https://docs.google.com/document/d/1PY06KZU97J-HU9nRArMH6biuoaCjPA_zm-EhIlTId2I/edit?usp=sharing) for interpretting results.



### **Problem #2.1: AWS Clarify & UCI's Adult Dataset**


#### **Steps #1-2: Load the data and import packages.**


This step has been completed in the first code cell.

#### **Step #3: Denote the facet column, the label column, and the group variable.**


Set the:
* Facet column as `Sex`.
* Label column with `Target` as the target column and `' >50K'` as the positive label.
* `Sex` as the group variable.

In [ ]:
facet_column = FacetColumn(# COMPLETE THIS LINE
label_column = LabelColumn(# COMPLETE THIS LINE
group_variable = # COMPLETE THIS LINE

#### **Step #4: Generate bias report.**

Check out the [cheat sheet](https://docs.google.com/document/d/1PY06KZU97J-HU9nRArMH6biuoaCjPA_zm-EhIlTId2I/edit?usp=sharing) to see how to interpret pre-training metrics. Using this page, interpret what the following report is saying.

<br>

**Run the cell below to generate your bias report.**

In [ ]:
report = bias_report(adult, facet_column, label_column, stage_type=StageType.PRE_TRAINING, group_variable=group_variable)
# use this to print your report - call it "report" for the code to work
for cl in report:
    print("\n\n","-"*35)
    print("-"*15, cl["value_or_threshold"], "-"*15)
    for metric in cl['metrics']:
        print(f"{metric['description']}: {metric['value']}")

#### **Step #5: Look at the imbalance.**

Use the cheat sheet to interpret what the bias report is telling you.

In [ ]:
# Write Male Results here

In [ ]:
# Write Female Results

### **Problem #2.2: AWS Clarify & UCI's Parkinson's Dataset**

This dataset is composed of a range of biomedical voice measurements from 42 people with early-stage Parkinson's disease recruited to a six-month trial of a telemonitoring device for remote symptom progression monitoring. The recordings were automatically captured in the patient's homes.

Columns in the table contain subject number, subject age, subject gender, time interval from baseline recruitment date, motor UPDRS, total UPDRS, and 16 biomedical voice measures. Each row corresponds to one of 5,875 voice recording from these individuals. The main aim of the data is to predict the motor and total UPDRS scores ('motor_UPDRS' and 'total_UPDRS') from the 16 voice measures.


#### **Steps #1-2: Load the data and import packages.**


In [ ]:
url = "https://raw.githubusercontent.com/the-codingschool/TRAIN-datasets/main/parkinsons/parkinsons.csv"
df = pd.read_csv(url)
df.head()

#### **Step #3: Denote the facet column, the label column, and the group variable.**

Set the:
* Facet column as `age`.
* Label column with `sex` as the target column and `'0'` as the positive label.
* `age` as the group variable.

In [ ]:
facet_column = FacetColumn(# COMPLETE THIS LINE
label_column = LabelColumn(# COMPLETE THIS LINE
group_variable = # COMPLETE THIS LINE

#### **Step #4: Generate bias report.**


Use the cheat sheet to interpret pre-training metrics. Using this page, interpret what the following report is saying.

<br>

**Run the cell below to generate your bias report.**

In [ ]:
report = bias_report(df, facet_column, label_column, stage_type=StageType.PRE_TRAINING, group_variable=group_variable)

# use this to print your report - call it "report" for the code to work
for cl in report:
    print("\n\n","-"*35)
    print("-"*15, cl["value_or_threshold"], "-"*15)
    for metric in cl['metrics']:
        print(f"{metric['description']}: {metric['value']}")

#### **Step #5: Look at the imbalance.**

Use the cheat sheet provided to interpret what the bias report is telling you.

#End of notebook
---
© 2024 The Coding School, All rights reserved